<a href="https://colab.research.google.com/github/statisticianinstilettos/Hackathon-Assets/blob/main/ODSC_Optimization_workshop_wildlife_video_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install opik

!pip install --upgrade opik opik-optimizer

In [2]:
!opik configure

Which Opik deployment do you want to log your traces to?
1 - Opik Cloud (default)
2 - Self-hosted Comet platform
3 - Local deployment

> 1
OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud
Please enter your Opik API key:
Do you want to use "statisticianinstilettos" workspace? (Y/n)Y
OPIK: Configuration saved to file: /root/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [6]:
import opik
from opik import track
from opik.integrations.openai import track_openai
from openai import OpenAI

os.environ["OPIK_PROJECT_NAME"] = "wildlife_video_project"

client = OpenAI()
tracked_client = track_openai(client)

In [7]:
#Research Agent

@track
def research_agent(animal: str, location: str, n_facts: int = 3) -> list[str]:
    """
    Generate short, engaging animal facts suitable for a wildlife video.
    """

    research_agent_prompt = f"""
    Generate {n_facts} short, fascinating facts about {animal} in {location}.
    Requirements:
    - Each fact must be 1 sentence
    - Written for a general audience
    - No emojis
    - No anthropomorphism
    """

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a wildlife education expert."},
            {"role": "user", "content": research_agent_prompt},
        ],
        temperature=0.7,
    )

    facts_text = response.choices[0].message.content
    facts = [f.strip("- ").strip() for f in facts_text.split("\n") if f.strip()]

    return facts

In [8]:
#Video Agent

@track
def video_agent(prompt: str) -> dict:
    """Generate a video using OpenAI's Sora model."""
    # Create video and wait for completion
    video = tracked_client.videos.create_and_poll(
        model="sora-2",
        prompt=prompt,
    )

    result = {"id": video.id, "status": video.status}

    # Download the video if generation succeeded
    # If OpenAI moderation rejected your request - you'll see it in the create_and_poll span output
    if video.status == "completed":
        content = tracked_client.videos.download_content(video_id=video.id)
        content.write_to_file("output_video.mp4")
        result["output_path"] = "output_video.mp4"

    return result


In [9]:
#Orchestration

@track
def agent_orchestrator(location: str, animal: str):

    facts = research_agent(animal, location)

    video_prompt = f"""
    Create a cinematic wildlife video of a {animal} in its natural habitat {location}.
    Narrate the video with the following facts:
    {facts}
    """

    video_result = video_agent(video_prompt)

    return {
        "animal": animal,
        "location": location,
        "facts": facts,
        "video": video_result
    }

In [10]:
# Generate final output
agent_orchestrator(location="New Mexico", animal="Elk")

{'animal': 'Elk',
 'location': 'New Mexico',
 'facts': ['1. Elk in New Mexico are the largest members of the deer family found in the state, with males weighing up to 700 pounds.',
  '2. During the fall rutting season, male elk produce loud bugling calls that can carry over long distances to attract females and establish dominance.',
  '3. New Mexico’s elk herds migrate seasonally, moving to higher elevations in summer and lower valleys in winter to find food and shelter.'],
 'video': {'id': 'video_69824ee97ed48191a9faad361a0b240e0e9400b3159e83a8',
  'status': 'completed',
  'output_path': 'output_video.mp4'}}

In [29]:
research_agent_system_prompt = f"""
    Generate 3 short, fascinating facts about the requested animal in the requested location.
    Requirements:
    - Each fact must be 1 sentence
    - Written for a general audience
    - No emojis
    - No anthropomorphism
    """

In [32]:
#Prompt Optimization


import opik
from opik.evaluation.metrics import Hallucination


client = opik.Opik()
dataset = client.get_or_create_dataset(name="wildlife-facts")
dataset.insert([
    {"question": "Roadrunner in New Mexico", "answer": "The greater roadrunner is New Mexico’s state bird and is commonly found in deserts and scrublands across the state. Roadrunners can run up to about 20 miles per hour, helping them catch prey like lizards, insects, and small snakes. Unlike many birds, roadrunners spend much of their time on the ground and are well adapted to hot, dry environments."},
    {"question": "Elk in Colorado", "answer": "Elk are found in the forests and mountain regions of northern and central New Mexico, especially in areas like the Jemez Mountains and Carson National Forest. Male elk grow large antlers each year that can span over four feet and are shed and regrown annually. Elk play an important ecological role by shaping vegetation patterns through their grazing and movement."},
    {"question": "Polar Bear in the Arctic", "answer": "Invisible Fur: Their fur is actually translucent and hollow, reflecting light to appear white over their jet-black skin. Sea Dwellers: They are the only bear species classified as marine mammals because they spend most of their lives on sea ice. Built-in Heaters: Their insulation is so effective that they often struggle more with overheating than with the Arctic cold."},
    {"question": "Panda in China", "answer": "Bamboo Metabolism: Giant Pandas possess a specialized pseudo-thumb evolved from a wrist bone specifically to help them strip and eat up to 84 pounds of bamboo daily. Ancient Lineage: Often called living fossils, pandas have existed as a distinct lineage for over 8 million years, remaining largely unchanged while other bear species evolved."},
    {"question": "Penguin in Antarctica", "answer": "Natural Tuxedos: Their black-and-white coloring is a camouflage called countershading, which hides them from predators looking down from above (blending with the dark water) and those looking up from below (blending with the bright sky).Deep Sea Divers: Emperor Penguins can dive to depths of over 500 meters and stay underwater for up to 20 minutes on a single breath.Saltwater Filtration: They possess a specialized supraorbital gland that acts as a filter, allowing them to drink saltwater by excreting the excess salt through their beaks."},
    {"question": "Coyote in Arizona", "answer": "Desert Trackers: They use their incredible sense of smell and hearing to detect prey moving deep underground or beneath the desert sand. Vocal Polyphony: A single coyote can sound like a whole pack due to the beau geste effect, where they rapidly shift their pitch and frequency to distort a listener's sense of their numbers. Suburban Adapters: Arizona coyotes have learned to exploit human environments so well that they can leap over 6-foot walls to navigate backyard landscapes."},
    {"question": "Red Fox in Virginia", "answer": "Magnetic Navigation: Red foxes use the Earth's magnetic field as a rangefinder to pinpoint the exact location of prey rustling under tall grass or snow. Tree Climbers: While they belong to the dog family, Virginia's red foxes have semi-retractable claws and cat-like agility that allow them to balance on low tree branches.Acoustic Hunters: Their hearing is so sensitive that they can hear a watch ticking from 40 yards away or a field mouse scurrying beneath several inches of soil."},
    {"question": "Desert Tortoise Las Vegas", "answer": "Here are three short, fascinating facts about the desert tortoise in the Las Vegas valley: Underground Architects: They spend up to 95% of their lives in deep underground burrows to escape the extreme Mojave heat and freezing winter nights."},
    {"question": "Hummingbird in California", "answer": "Energy Dynamos: To power their rapid wingbeats, California’s hummingbirds have the highest metabolic rate of any homeothermic animal, requiring them to visit up to 2,000 flowers a day. Seismic Navigators: They possess specialized sensors in their talons that allow them to detect the subtle tectonic shifts of the San Andreas Fault hours before a human can feel an earthquake.Torpor Masters: On cold California nights, they can drop their body temperature from 40°C to near-ambient levels to save energy, entering a state of deep sleep called torpor."},
    {"question": "Mountain Lion in California", "answer": "Incredible Leapers: A mountain lion can jump up to 18 feet vertically and leap 45 feet horizontally to navigate the steep terrain of the Sierra Nevada. Invisible Neighbors: Despite being the state's top predator, they are so elusive that researchers estimate a human can hike within 20 feet of one without ever realizing it is there.Telepathic Tracking: Through a unique adaptation in their frontal lobe, California mountain lions can mentally map the exact heartbeats of every deer within a three-mile radius to plan their ambush."},
    ])
def answer_quality(item, output):
    metric = Hallucination()
    return metric.score(reference=item["answer"], output=output)

In [31]:
from opik_optimizer import MetaPromptOptimizer, ChatPrompt

prompt = ChatPrompt(
    messages=[
        {"role": "system", "content": research_agent_system_prompt},
        {"role": "user", "content": "{question}"},
    ],
    model="openai/gpt-4o-mini"  # The model your prompt runs on
)

optimizer = MetaPromptOptimizer(model="openai/gpt-4o")  # The model that improves your prompt
result = optimizer.optimize_prompt(
    prompt=prompt,
    dataset=dataset,
    metric=answer_quality,
    max_trials=3,
    n_samples=10,
)

result.display()


╭──────────────────────────────────────────────────────────────────────────────╮
│ ● Running Opik Evaluation - MetaPromptOptimizer                              │
│                                                                              │
│ -> View optimization details ]8;id=541450;https://www.comet.com/opik/api/v1/session/redirect/optimizations/?optimization_id=019c2537-de77-761a-b237-3889d63a6b8b&dataset_id=019c250c-e890-77b3-ba53-edc85e718a36&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\                          │
╰──────────────────────────────────────────────────────────────────────────────╯

> Let's optimize the prompt:

╭─ system ─────────────────────────────────────────────────────────────────────╮
│                                                                              │
│                                                                              │
│      Generate 3 short, fascinating facts about the requested animal in the   │
│  requested location.                                                         │
│      Requirements:                                                           │
│      - Each fact must be 1 sentence                                          │
│      - Written for a general audience                                        │
│      - No emojis                                                             │
│      - No anthropomorphism                                                   │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭─ user ───────────────────────────────────────────────────────────────────────╮
│                                                                              │
│  {question}                                                                  │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

Using MetaPromptOptimizer with the parameters:

...

│

│

> Optimization complete

  No improvement over the initial prompt.

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:             MetaPromptOptimizer                                                                      ║
║ Dataset ID:            019c250c-e890-77b3-ba53-edc85e718a36                                                     ║
║ Model Used:            openai/gpt-4o                                                                            ║
║ Metric Evaluated:      answer_quality                                                                           ║
║ Initial Score:         0.0000                                                                                   ║
║ Final Best Score:      0.0000                                                                                   ║
║ Total Improvement:     0.00% (no improvement from 0)                                                            ║
║ Key metrics:           Best: 0.0000 | Δ: N/A | Stop: Budget                                                     ║
║ Statistics:            Trials: 3 | Rounds: 1 | Candidates: 3                                                    ║
║ Best candidate:        round0_cand1                                                                             ║
║ Best prompt summary:   system + user                                                                            ║
║ Optimization run link: ]8;id=163030;https://www.comet.com/opik/api/v1/session/redirect/optimizations/?optimization_id=019c2537-de77-761a-b237-3889d63a6b8b&dataset_id=019c250c-e890-77b3-ba53-edc85e718a36&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\Open in Opik Dashboard]8;;\                                                                   ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭────────────────────────────────────────── Final Optimized Prompt ───────────────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  ╭─ system ─────────────────────────────────────────────────────────────────────╮                           │ ║
║ │  │                                                                              │                           │ ║
║ │  │                                                                              │                           │ ║
║ │  │      Generate 3 short, fascinating facts about the requested animal in the   │                           │ ║
║ │  │  requested location.                                                         │                           │ ║
║ │  │      Requirements:                                                           │                           │ ║
║ │  │      - Each fact must be 1 sentence                                          │                           │ ║
║ │  │      - Written for a general audience                                        │                           │ ║
║ │  │      - No emojis                                                             │                           │ ║
║ │  │      - No anthropomorphism                                                   │                           │ ║
║ │  │                                                                              │                           │ ║
║ │  │                                                                              │                           │ ║
║ │  ╰──────────────────────────────────────────────────────────────────────────────╯                           │ ║
║ │  ╭─ user ───────────────────────────────────────────────────────────────────────╮                           │ ║
║ │  │                                           